In [1496]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [1497]:
data_1_sheet_kmg = pd.read_excel(r"C:\Users\samen\datasheet1.xlsx",sheet_name="GS-KS-MS data sheet 1")

In [1498]:
try:
    data_1_sheet_kmg.drop('Outlet tagging',axis=1,inplace=True)
except:
    pass

In [1499]:
data_1_sheet_kmg=pd.DataFrame(data_1_sheet_kmg)

In [1500]:
data_1_sheet_kmg.shape

(22193, 9)

# KPI BEGIN

In [1502]:
city_category = list(data_1_sheet_kmg['city'].unique())
city_category
len_city_category = len(city_category)

In [1503]:
len_customer = len(data_1_sheet_kmg['customer'].unique())

In [1504]:
city_category

['TRIVANDRUM', 'TRISSUR']

In [1505]:
re = list(data_1_sheet_kmg['re'].unique())

In [1506]:
re


['MS', 'KS', 'GS']

In [1507]:
i=0
count=0
name="dataframe_new"

In [1508]:
d={}

In [1512]:
for c in city_category:
    sheet_1_tvm = data_1_sheet_kmg[data_1_sheet_kmg['city']==c]
    for i in range(len(re)):
        print("re=" + re[i]+ str(' running'))
        
        tvm_ks = sheet_1_tvm[sheet_1_tvm['re']==re[i]]
        tvm_ks_customer = list(tvm_ks['customer'].unique())
        month_cat = list(tvm_ks['mth'].unique())
        sum_df = []
        sum_ppg=[]
        call_sum = []
        cat = []
        #print(tvm_ks['mth'])
        #mth=[]
        m=[]
        m_df = pd.DataFrame(columns=month_cat)
        
        for k in tvm_ks_customer:
            sum_df.append(int(round(tvm_ks.loc[tvm_ks['customer']==k,'sales_value'].sum())))
            sum_ppg.append(tvm_ks.loc[tvm_ks['customer']==k,'uniq_npg'].sum())
            call_sum.append(tvm_ks.loc[tvm_ks['customer']==k,'invoices'].sum())
            for mt in month_cat:
                mth=tvm_ks.loc[(tvm_ks['mth']==mt) &(tvm_ks['customer']==k),'sales_value'].sum()
                #m["{}".format(mt)]=mth
                m.append(mth)
            m=pd.Series(m,index=m_df.columns)
            m_df = m_df.append(m,ignore_index=True)
            m=[]
            month_cat = list(tvm_ks['mth'].unique())
            #m_sum_t = 
            
                
                #print(mth)
                #quit()
                
        tvm_ks_final = pd.DataFrame()
        #print(c)
        tvm_ks_final['customer'] = tvm_ks_customer
        city = [str(c) for i in range(tvm_ks_final.shape[0])]
        tvm_ks_final['City'] = city
        
        tvm_ks_final['sum_of_sales_value'] = sum_df
        print("here")
        print(i)
        tvm_ks_final['re'] = str(re[i])
        sum_ppg = [round(x/7,2) for x in sum_ppg ]
        tvm_ks_final['Total unique PPGs/7'] = sum_ppg
        call_sum = [round(x/7,2) for x in call_sum]
        tvm_ks_final['Total no. of calls/7'] = call_sum
        tvm_ks_final['Percentile_rank']=round(tvm_ks_final['sum_of_sales_value'].rank(pct=True),2)
        #tvm_ks_final['months']=mth
        new_list = tvm_ks_final['Percentile_rank'].to_list()
        for x in new_list:
            if x<=0.3:
                cat.append('low')
            elif x>0.3 and x<0.7:
                cat.append('medium')
            else:
                cat.append('high')

        tvm_ks_final['category'] = cat
        tvm_ks_final = pd.concat([tvm_ks_final,m_df],axis=1)
        d["{}_{}".format(c,re[i])]=pd.DataFrame(tvm_ks_final)
        count+=1
        d["{}_{}".format(c,re[i])].fillna(0,inplace=True)
        
        #pd.concat([df3, df4], axis='col')
        
        
        
    
    #pd.DataFrame(tvm_ks_final)
    
   
    
    
    

    
    

re=MS running
here
0
re=KS running
here
1
re=GS running
here
2
re=MS running
here
0
re=KS running
here
1
re=GS running
here
2


In [1513]:
month_cat = list(tvm_ks['mth'].unique())

In [1514]:
month_cat

[8, 12, 9, 1, 2, 11, 10]

In [1515]:
#tvm_ks_final

In [1516]:
keys_final= []
print("The Keys Are :- ")
print("use d[keyname to pick up  a dataframe]")
print("      ")
for key in d.keys():
    keys_final.append(key)
    print(key)

The Keys Are :- 
use d[keyname to pick up  a dataframe]
      
TRIVANDRUM_MS
TRIVANDRUM_KS
TRIVANDRUM_GS
TRISSUR_MS
TRISSUR_KS
TRISSUR_GS


In [1517]:
keys_final

['TRIVANDRUM_MS',
 'TRIVANDRUM_KS',
 'TRIVANDRUM_GS',
 'TRISSUR_MS',
 'TRISSUR_KS',
 'TRISSUR_GS']

# MONTH DICT + JOINING OF OUTLET TAGGING

In [1520]:
month_dict = {8:'aug',
              9:'sep',
              10:'oct',
              11:'nov',
              12:'dec',
              1:'jan',
              2:'feb',
              3:'mar',
              4:'april',
              5:'may',
              6:'jun',
              7:'july'
}

In [1521]:
cat_sheet =pd.DataFrame() #category of unique values ! 

In [1522]:
for di in keys_final:
    new = d[di][['customer','category']]
    cat_sheet = cat_sheet.append(new)
# setting a dictionary !     

In [1523]:
cat_sheet.shape[0]==len(data_1_sheet_kmg['customer'].unique()) #testing shapes of unqiue value ! 


True

In [1524]:
cat_dict  = dict(zip(cat_sheet['customer'],cat_sheet['category'])) #creating Dict for values to apply to main dataset ! 

In [1525]:
data_1_sheet_kmg['category'] = data_1_sheet_kmg['customer'].map(cat_dict)
#setting the values back to the table!!!

In [1528]:
final_dict={} #empty dict to create and store the needed tables , since we dont know how many might come in! 

# DIct and Adding Category Complete ! 


# Creating Final Set for DataSheet1 - Currently for TVM

In [1530]:
bon = ['avg_sale_value_per_retailer','no_of_outlets','median_sale_value_per_outlet','avg_npgs_per_retailer','median_npgs_per_outlet','avg_no_of_calls_per_outlet','median_no_of_calls_per_outlet']
# 7 value tables that are calculated

In [1531]:
re

['MS', 'KS', 'GS']

In [1532]:

city_category

['TRIVANDRUM', 'TRISSUR']

No of Outlets 

In [1533]:
from statistics import median
import numpy as np

In [1535]:
category_valu = ['low','medium','high']

In [1536]:
var_run=0

In [1537]:
for c_name in city_category:
    bcount=0
    keys_final=[c_name+str("_")+str(i) for i in re]
    rows =[]
    new_1 = []
    final_df_1  = pd.DataFrame()
    category_valu=['low','medium','high']
       
    #Need 7 outputs ! 

    #avg sale value per retailer
    for x in re:
        column1='avg sale per retailer'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_1 = pd.DataFrame(columns = columns)
    final_df_1.insert(0,column1,rows)
    rows=[]
    print(final_df_1.shape)

    #no of outlets    
    for x in re:
        column1='number of outlets'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_2 = pd.DataFrame(columns = columns)
    final_df_2.insert(0,column1,rows)
    rows=[]

    print(final_df_2.shape)

    #median sale value per outlet
    for x in re:
        column1='median sale value per outlet'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_3 = pd.DataFrame(columns = columns)
    final_df_3.insert(0,column1,rows)
    rows=[]
    print(final_df_3.shape)


    #avg npgs per retailer
    for x in re:
        column1='avg npg per retailer'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_4 = pd.DataFrame(columns = columns)
    final_df_4.insert(0,column1,rows)
    rows=[]
    print(final_df_4.shape)




    #median npgs per outlet
    for x in re:
        column1='median npg per outlet'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_5 = pd.DataFrame(columns = columns)
    final_df_5.insert(0,column1,rows)
    rows=[]
    print(final_df_5.shape)





    #avg no of calls per outlet
    for x in re:
        column1='avg call per retailer'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_6 = pd.DataFrame(columns = columns)
    final_df_6.insert(0,column1,rows)
    rows=[]
    print(final_df_6.shape)




    #median no of calls per outlet
    for x in re:
        column1='median call per outlet'
        columns = [month_dict[m] for m in month_cat]
        new_1.append(str(x)+"_"+'low')
        new_1.append(str(x)+"_"+'mid')
        new_1.append(str(x)+"_"+'high')
        #new_1.append(str(x)+"_"+'overall')
        rows.append(new_1)
        new_1=[]
    rows = [item for sublist in rows for item in sublist] #flattening the list of list
    #df.insert(0,'name_of_column',value)   
    final_df_7 = pd.DataFrame(columns = columns)
    final_df_7.insert(0,column1,rows)
    rows=[]
    print(final_df_7.shape)
    print("empty Cells Created")
    
    
    
    ##########################OUTPUT SECTION#########################################
    #print(c)
    #avg sale value per retailer
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
                #print(m)
                len_ = len(test.loc[test[m]==0]) #finidng unadded values
                nval = test[m].sum()/(test.shape[0]-len_)
                #print(nval)
                final_df_1[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_1.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_1)
    bcount+=1

    
    
    #no of outlets
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
                #print(m)
                #len_ = len(test.loc[test[m]==0])
                #print(len_)#finidng unadded values
                nval = len(test.loc[test[m]!=0])
                #print(nval)
                final_df_2[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_2.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_2)
    bcount+=1
    
    
    #median sale value per outlet
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
               # print(m)
                #len_ = len(test.loc[test[m]==0])
                #print(len_)#finidng unadded values
                new = test.loc[test[m]!=0]
                nval =median(new[m])



                #print(nval)
                final_df_3[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_3.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_3)
    bcount+=1
    
    
    #avg npgs per retailer
    import numpy as np
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
                #print(m)
                #len_ = len(test.loc[test[m]==0])
                #print(len_)#finidng unadded values
                test1 = test.loc[test[m]!=0]
                nval = np.ceil(test1['Total unique PPGs/7'].sum()/test.shape[0])
                
                #print(nval)
                final_df_4[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_4.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_4)
    bcount+=1

    #median npgs per outlet
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
                #print(m)
                #len_ = len(test.loc[test[m]==0])
                #print(len_)#finidng unadded values
                test1 = test.loc[test[m]!=0]
                nval = np.ceil(median(test1['Total unique PPGs/7']))
                

                #print(nval)
                final_df_5[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_5.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_5)
    bcount+=1

    #avg calls per retailer
    import numpy as np
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
                #print(m)
                #len_ = len(test.loc[test[m]==0])
                #print(len_)#finidng unadded values
                test1 = test.loc[test[m]!=0]
                nval = np.ceil(test1['Total no. of calls/7'].sum()/test.shape[0])
                #print(nval)
                final_df_6[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_6.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_6)
    bcount+=1
    
    #median calls per outlet
    import numpy as np
    i=0
    for k in keys_final:
        #print(k)
        for c in category_valu:
            #print(c)
            test = d[k].loc[d[k]['category']==c]
            #print(test)

            for m in month_cat:
                #print(m)
                #len_ = len(test.loc[test[m]==0])
                #print(len_)#finidng unadded values
                test1 = test.loc[test[m]!=0]
                nval = round(median(test1['Total no. of calls/7']))

                #print(nval)
                final_df_7[month_dict[m]][i]=nval
                #print(i)
                #i+=1
            i+=1
    final_df_7.fillna(method='bfill',inplace=True)
    final_dict["{}_{}".format(c_name,bon[bcount])]=pd.DataFrame(final_df_7)
    bcount+=1
  

(9, 8)
(9, 8)
(9, 8)
(9, 8)
(9, 8)
(9, 8)
(9, 8)
empty Cells Created
(9, 8)
(9, 8)
(9, 8)
(9, 8)
(9, 8)
(9, 8)
(9, 8)
empty Cells Created


In [1562]:
#Creating a list of keys for the firts one ! 

sheet1_dict =[]
sheet1_dict = [x for x in final_dict.keys()]

In [1564]:
data_2_sheet_kmg = pd.read_excel(r"C:\Users\samen\datasheet2.xlsx",sheet_name="Data sheet 2")

In [1565]:
data_2_sheet_kmg=pd.DataFrame(data_2_sheet_kmg)

In [2118]:
try:
    data_2_sheet_kmg.drop('Outlet tagging',axis=1,inplace=True)
    data_2_sheet_kmg.drop(['TRIVANDRUM|GS PPG flag','TRIVANDRUM|KS PPG flag','TRIVANDRUM|MS PPG flag','TRISSUR|GS PPG flag','TRISSUR|KS PPG flag','TRISSUR|MS PPG flag'],axis=1,inplace=True)
except:
    pass

In [2120]:
product_dict = {}

In [2121]:
product_dict  = dict(zip(data_2_sheet_kmg['npg_code'],data_2_sheet_kmg['npg_desc'])) #product Dictionary

In [2123]:
npg_list= [x for x in product_dict.keys()]


In [2125]:
data_2_sheet_kmg['category'] = data_2_sheet_kmg['customer'].map(cat_dict) # applying outlet tagging ! 

In [2126]:
city_cat_2 = list(data_2_sheet_kmg['city'].unique())

In [2127]:
re_2 = list(data_2_sheet_kmg['re'].unique())

In [2129]:
re_2_dict = {}

In [2130]:
daf_2 = pd.DataFrame(data_2_sheet_kmg.loc[(data_2_sheet_kmg['re']=='MS') & (data_2_sheet_kmg['city']=='TRISSUR')])
    

Initial splits on first tree of city's then on the basis of different Re's for each and adding them to a dictionary

In [2131]:
for city in city_cat_2:
    for r in re_2:
        daf_2 = pd.DataFrame(data_2_sheet_kmg.loc[(data_2_sheet_kmg['re']==r)& (data_2_sheet_kmg['city']==city)])
        re_2_dict['{}_{}'.format(city,r)]=pd.DataFrame(daf_2)
#Dictionary basedd on city+re splits with entire data!         

In [2132]:
re_2_keys=[]

In [2133]:
for key in re_2_dict.keys():
    re_2_keys.append(key)
re_2_keys

['TRIVANDRUM_MS',
 'TRIVANDRUM_KS',
 'TRIVANDRUM_GS',
 'TRIVANDRUM_WS',
 'TRIVANDRUM_MT',
 'TRIVANDRUM_CS',
 'TRIVANDRUM_PS',
 'TRISSUR_MS',
 'TRISSUR_KS',
 'TRISSUR_GS',
 'TRISSUR_WS',
 'TRISSUR_MT',
 'TRISSUR_CS',
 'TRISSUR_PS']

Extraction process for sales value

In [2178]:
sale_dict = {}


In [2179]:
for rk in re_2_keys:
    temp =re_2_dict[rk]
    temp = temp.dropna()
    #npg_list = list(temp['npg_code'].values)
    rows = npg_list
    columns = [str(rk)+'01',str(rk)+'02',str(rk)+'03',str(rk)+'04',str(rk)+'05',str(rk)+'06',str(rk)+'07']
    column1 = 'sale_dict-npg'
    model = pd.DataFrame(columns=columns)
    model.insert(0,column1,rows)
    i=0
    for ni in npg_list:

        temp1 = temp.loc[temp['npg_code']==ni]
        model[str(rk)+'01'][i]=temp1.loc[temp['SV_01']!=0,'SV_01'].sum()
        model[str(rk)+'02'][i]=temp1.loc[temp['SV_02']!=0,'SV_02'].sum()
        model[str(rk)+'03'][i]=temp1.loc[temp['SV_03']!=0,'SV_03'].sum()
        model[str(rk)+'04'][i]=temp1.loc[temp['SV_04']!=0,'SV_04'].sum()
        model[str(rk)+'05'][i]=temp1.loc[temp['SV_05']!=0,'SV_05'].sum()
        model[str(rk)+'06'][i]=temp1.loc[temp['SV_06']!=0,'SV_06'].sum()
        model[str(rk)+'07'][i]=temp1.loc[temp['SV_07']!=0,'SV_07'].sum()
        i+=1
    sale_dict["{}".format(rk)]=pd.DataFrame(model)
    i=0


In [2180]:
sale_dict.keys()

dict_keys(['TRIVANDRUM_MS', 'TRIVANDRUM_KS', 'TRIVANDRUM_GS', 'TRIVANDRUM_WS', 'TRIVANDRUM_MT', 'TRIVANDRUM_CS', 'TRIVANDRUM_PS', 'TRISSUR_MS', 'TRISSUR_KS', 'TRISSUR_GS', 'TRISSUR_WS', 'TRISSUR_MT', 'TRISSUR_CS', 'TRISSUR_PS'])

In [2182]:
product_dict['CP1111CODCX03']

'TP Col DenCrm Rs.20'

Extraction process for Quantity

In [2183]:
quant_dict = {}


In [2184]:
for rk in re_2_keys:
    temp =re_2_dict[rk]
    temp = temp.dropna()
    #npg_list = list(temp['npg_code'].unique())
    rows = npg_list
    columns = [str(rk)+'01',str(rk)+'02',str(rk)+'03',str(rk)+'04',str(rk)+'05',str(rk)+'06',str(rk)+'07']
    column1 = 'quant_dict-npg'
    model = pd.DataFrame(columns=columns)
    model.insert(0,column1,rows)
    i=0
    for ni in npg_list:

        temp1 = temp.loc[temp['npg_code']==ni]
        model[str(rk)+'01'][i]=int(temp1.loc[temp['QY_01']!=0,'QY_01'].sum())
        model[str(rk)+'02'][i]=int(temp1.loc[temp['QY_02']!=0,'QY_02'].sum())
        model[str(rk)+'03'][i]=int(temp1.loc[temp['QY_03']!=0,'QY_03'].sum())
        model[str(rk)+'04'][i]=int(temp1.loc[temp['QY_04']!=0,'QY_04'].sum())
        model[str(rk)+'05'][i]=int(temp1.loc[temp['QY_05']!=0,'QY_05'].sum())
        model[str(rk)+'06'][i]=int(temp1.loc[temp['QY_06']!=0,'QY_06'].sum())
        model[str(rk)+'07'][i]=int(temp1.loc[temp['QY_07']!=0,'QY_07'].sum())
        i+=1
    quant_dict["{}".format(rk)]=pd.DataFrame(model)
    i=0


In [2185]:
quant_dict.keys()

dict_keys(['TRIVANDRUM_MS', 'TRIVANDRUM_KS', 'TRIVANDRUM_GS', 'TRIVANDRUM_WS', 'TRIVANDRUM_MT', 'TRIVANDRUM_CS', 'TRIVANDRUM_PS', 'TRISSUR_MS', 'TRISSUR_KS', 'TRISSUR_GS', 'TRISSUR_WS', 'TRISSUR_MT', 'TRISSUR_CS', 'TRISSUR_PS'])

In [2187]:
product_dict['CP1111CODCX03']

'TP Col DenCrm Rs.20'

Extraction process for Participation

In [2188]:
part_dict = {}


In [2189]:
for rk in re_2_keys:
    temp =re_2_dict[rk]
    temp = temp.dropna()
    cust_count = len(temp['customer'].unique())
    #npg_list = list(temp['npg_code'].unique())
    rows = npg_list
    columns = [str(rk)+'01%',str(rk)+'02%',str(rk)+'03%',str(rk)+'04%',str(rk)+'05%',str(rk)+'06%',str(rk)+'07%']
    column1 = 'part_dict-npg'
    model = pd.DataFrame(columns=columns)
    model.insert(0,column1,rows)
    i=0
    for ni in npg_list:

        temp1 = temp.loc[temp['npg_code']==ni]
        model[str(rk)+'01%'][i]=round((temp1.loc[temp['M_01']=='Y','M_01'].count()/cust_count)*100,2)
        model[str(rk)+'02%'][i]=round((temp1.loc[temp['M_02']=='Y','M_02'].count()/cust_count)*100,2)
        model[str(rk)+'03%'][i]=round((temp1.loc[temp['M_03']=='Y','M_03'].count()/cust_count)*100,2)
        model[str(rk)+'04%'][i]=round((temp1.loc[temp['M_04']=='Y','M_04'].count()/cust_count)*100,2)
        model[str(rk)+'05%'][i]=round((temp1.loc[temp['M_05']=='Y','M_05'].count()/cust_count)*100,2)
        model[str(rk)+'06%'][i]=round((temp1.loc[temp['M_06']=='Y','M_06'].count()/cust_count)*100,2)
        model[str(rk)+'07%'][i]=round((temp1.loc[temp['M_07']=='Y','M_07'].count()/cust_count)*100,2)
        i+=1
    part_dict["{}".format(rk)]=pd.DataFrame(model)
    i=0


c:\users\samen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in long_scalars
  from ipykernel import kernelapp as app
c:\users\samen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in long_scalars
  app.launch_new_instance()
c:\users\samen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars
c:\users\samen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in long_scalars
c:\users\samen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in long_scalars
c:\users\samen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_sca

In [2190]:
part_dict.keys()

dict_keys(['TRIVANDRUM_MS', 'TRIVANDRUM_KS', 'TRIVANDRUM_GS', 'TRIVANDRUM_WS', 'TRIVANDRUM_MT', 'TRIVANDRUM_CS', 'TRIVANDRUM_PS', 'TRISSUR_MS', 'TRISSUR_KS', 'TRISSUR_GS', 'TRISSUR_WS', 'TRISSUR_MT', 'TRISSUR_CS', 'TRISSUR_PS'])

In [2192]:
product_dict['CP1111CODCX11']

'TP Col DenCrm 200gX2+100g'

# Fetching of [Sales value, Participation, quantity for each] and stored in respective dicts

In [2194]:
final_sale_df = pd.DataFrame()

In [2195]:
for key in list(sale_dict.keys()):
    final_sale_df = pd.concat([final_sale_df,sale_dict[key]],axis=1)

quantity values dict

In [2197]:
final_quant_df = pd.DataFrame()

In [2198]:
for key in list(quant_dict.keys()):
    final_quant_df = pd.concat([final_quant_df,quant_dict[key]],axis=1)

participation value dict

In [2200]:
final_part_df = pd.DataFrame()

In [2201]:
for key in list(part_dict.keys()):
    final_part_df = pd.concat([final_part_df,part_dict[key]],axis=1)

# Getting the Top 20 from each of the above - sales,quant,part

initial modifications